# MLiP IsolationForest am Beispiel PSF

#### In diesem Notebook wird der IsolationForest anhand des Anwendungsbeispiels Profilschienenführung (PSF) geübt. 

Linearführungen und Profilschienenführungen sind für einen nicht unbedeutenden Teil der Ausfälle bei Werkzeugmaschinen.
Typische Fehlerfälle die dabei auftreten sind Mangelschmierungen, Pittings an Laufbahnen oder an den Wälzkörpern. In diesem Beispiel ist aber der grundlegende Zustand der PSF von Interesse, d.h. ist der Zustand fehlerfrei (OK) oder machen sich Verschleißerscheinungen (Ausfall) deutlich. 

Für dieses Beispiel wurden mehrere Profilschienen unter Belastung bis zu ihrem Lebensende verfahren. Die dabei auftretenden Fehler sind natürlich entstanden. Dementsprechend unterscheiden Sie sich sowohl in der Art als auch in der Stärke.

Während der Versuche wurde mittels einem 3-achsigen MEMS-Sensor die Beschleunigungen, in Verfahrrichtung (Acc_X)
in Richtung oder entgegen Erdmittelpunkt (Acc_Y) und orthogonal zu beiden in Richtung des Seitennormalenvektor des Versuchsstands (Acc_Z) aufgenommen. Die entsprechenden Zeitreihen:
<table><tr>
<td> <img src="MLiP_PSF_Messfahrt_gut.png" alt="Status OK" style="width: 350px;"/> </td>
<td> <img src="MLiP_PSF_Messfahrt_ausfall.png" alt="Status Ausfall" style="width: 350px;"/> </td>
</tr></table>
Für diese Aufgabe wurde nur die Beschleunigung in Verfahrrichtung (Acc_X) verwendet. Deren Werte wurden bereits zu Features zusammengefasst, anhand denen nun ein Isolation Forest angewendet wird.


### Data-Mining-Prozess:

![Bild konnte nicht geladen werden! 1. Daten erfassen - 2. Daten erkunden - 3. Daten vorbereiten - 4. Modelle bilden - 5. Modelle validieren - 6. Modell testen](Prozess_Modellentwicklung_v2.png "ML Vorgehen")

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

(Wähle für alle Notebooks in allen Übungen immer die gleiche Zahl (z.B. den Tag deines Geburtstags), dann sind die Ergebnisse der verschiedenen Machine-Learning-Verfahren vergleichbar da dann alle Notebooks mit der "gleichen" Folge an Zufallszahlen arbeiten)

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 1. Daten erfassen - Daten importieren

Import der Daten mittels der read_csv-Funktion von Pandas.  
Achtung, es gibt zwei Datensätze, einen mit den Trainingsdaten und einen mit den Testdaten. 

In [ ]:
# Lade Datensatz
df = pd.read_csv("CM_PSF_Features.csv")

### 2. Daten erkunden

In [ ]:
# Datensatz anzeigen
df.head(10)

In [ ]:
# Datensatz beschreiben
df.describe()

In [ ]:
# Berechnung der No Information Rate (NIR)
# also welche Genauigkeit wird erhalten, wenn immer ok vorhegesagt wird. 

NIR = len(df[df.Status=='OK'])/len(df)

print('NIR / baseline für Bewertung der Accuracy: ' + str(round(NIR, 4)))

In [ ]:
# Visualisierung Klassenzugehörigkeits-Matrix für zufällige Auswahl von 200 Datenpunkten
import seaborn as sns

sns.pairplot(df.sample(n=200), hue="Status")
plt.show()

### 3.1 Daten vorbereiten - Encoding der Label

Die Zielspalte liegt noch im String Format vor, damit können die Verfahren nicht umgehen. Daher wird mit der Funktion LabelBinarizer von sklearn die String Werte 'OK' und 'Ausfall' in Zahlenwerte 1 und -1 umgewandelt. 

In [ ]:
# Encoding des Status mittels dem LabelBinarizer
# Import des LabelBinarizer
from sklearn.preprocessing import LabelBinarizer

# Initialsierung
le = LabelBinarizer(neg_label=-1)

# LabelBinarizer bekommt zu transformierende Labels, erstellt internes Mapping
le.fit(['OK', 'Ausfall'])

# Beispielhafte Anwendung des LabelBinarizer
print('Beispiel LabelBinarizer')
print('Original Labels: \t' + str(['Ausfall', 'OK', 'OK', 'Ausfall', 'OK']))
print('Transformierte Labels: \t' + str(le.transform(['Ausfall', 'OK', 'OK', 'Ausfall', 'OK'])))

# Anwendung LabelBinarizer auf DataFrame
df = df.assign(status_enc = le.transform(df['Status']) )
df.head(5)

### 3.2 Daten vorbereiten - Aufteilung der Daten

Einzelne Prüflinge können spezifische Charakteristika aufweisen. Daher kann es passieren, dass der ML Algorithmus diese Charakteristika und nicht die grundlegenden Zusammenhalte lernt. Durch die Bildung eines vollständig unabhängigen Testdatensatz wird sichergestellt, dass das Modell wirklich gut auf neue Versuche generalisiert. 

In [ ]:
# Daten in Trainings- und Test aufteilen

# Setting random seed for numpy
np.random.seed(my_seed)

# zufällige Auswahl von 6 Prüflingen für den Testdatensatz
test_IDs = np.random.choice(df['Pruefling_ID'].unique(),6)
print('Ausgewählte Prüflinge für Testdatensatz: ' + str(test_IDs))

# Aufteilung in Trainings und Testdatensatz
df_test = df[df['Pruefling_ID'].isin(test_IDs)]
df_train = df[~df['Pruefling_ID'].isin(test_IDs)]

# Aufteilung in X und y jeweils für Trainings- und Testdaten
X_train = df_train.drop(columns=['Messfahrt_ID', 'Pruefling_ID', 'Status', 'status_enc'])
y_train = df_train['status_enc']

X_test = df_test.drop(columns=['Messfahrt_ID', 'Pruefling_ID', 'Status', 'status_enc'])
y_test = df_test['status_enc']

# Ausgabe Datensätze und Anzahl Datenpunkte
print("\nAnzahl Traingsdaten: \t\t" + str(len(y_train)) + " / " + str(len(df)))
print("Anzahl Testdaten: \t\t" + str(len(y_test)) + " / " + str(len(df)))

### 3.3 Daten vorbereiten - Normierung der Daten
__Optional__ können die Input-Parameter normiert werden.   
Allerdings hat dies keine Auswirkungen auf die Performance der Entscheidungsbaum-basierten Verfahren. 
Daher wird hier auf diesen Schritt verzichtet.

### 4.1 Modell bilden - Modell importieren

In [ ]:
# Importiere das Modell aus der Bibliothek sklearn
from sklearn.ensemble import IsolationForest

### 4.2 Modelle bilden - Hyperparameter Tuning mittels Gittersuche

Um die optimalen Werte für die Hyperparameter zu finden, wird eine Gittersuche mit Crossvalidation __GridSearchCV__ durchgeführt:
1. Hyperparameter bestimmen\
Für die Ensemble Learning Verfahren RandomForest und GradientBoosting werden in diesem Notebook 3 Hyperparameter eingestellt:
- n_estimators: Anzahl der Bäume, Suchbereich Liste [50, 100, 150, 200]
- contamination: Anteil der Ausreißer die im Datensatz angenommen werden, Suchbereich Liste [0.01, 0.03, 0.05, 0.07]
- max_features: Maximale Anzahl Features je Baum, Suchbereich\
Die Hyperparameter werden als Dictionary gebraucht, das ist eine Datenstruktur mit mehreren Einträgen von key-Value Paaren, d.h. eindeutige Bezeichner (keys) und den zugehörigen Werten.


2. Modell erstellen\
Wenn manche Hyperparameter nur einen Wert annehmen sollen, so kann man diese entweder als Hyperparameter in Schritt 1 mit aufnehmen oder direkt an das Modell übergeben. 

3. Gittersuche parameterieren\
In diesem Schritt werden Parameter für die Klasse GridSearchCV eingestellt und dann in einer Variable gespeichert.\
Parameter: 
* 1. Parameter: das ML-Modell, hier model 
* 2. Parameter: die Hyperparameter, hier hyper_parameters, diese müssen als dictionary gegeben sein (vgl. Schritt 1) 
* return_train_score=True, um die Trainingsergebnisse zu erhalten
* cv=5, Anzahl der Folds, die gebildet werden 
* n_jobs=-2 Parallelisierung der Berechnung, alle bis auf ein Prozessorkern wird genutzt

4. Modelle trainieren\
Das Training der Modelle erfolgt wieder mit dem fit Befehl. Diesem müssen die Trainingsdaten übergeben werden. 

TODO:
- Erzeuge ein Modell, Random Forest oder Gradien Boosting Model, je nach Wunsch
- Übergebe der GridSearchCV Klasse, das Modell (model) und die Hyperparameter (hyper_parameters)
- Setze die Trainingsdaten in den .fit-Befehle ein (X_train und y_train)  
    
AUSGABE:
- Anzahl der getesteten Hyperparameterkombinationen
- Zeitdauer für Gittersuche
- bestes Modell


In [ ]:
# Erstellung und Training der Modelle 

# Werte für die Gittersuche/Hyperparameter
from sklearn.model_selection import GridSearchCV
import time

start_timer = time.monotonic()
# 1. Hyperparameter bestimmen - für die Suche und Suchbereiche 
hyper_parameters = {"n_estimators": [50, 100, 150, 200], 
                    "contamination":  [0.03, 0.04, 0.05, 0.06, 0.07], 
                    "max_features" : [0.7, 0.8, 0.9, 1]}

# 2. Modell erzeugen
model = IsolationForest(random_state=my_seed)

# Achtung GridSearch will Modelle, die eine Scoring Funktion besitzen. 
# Da der IsolationForest keine besitzt, wird eine Scoring Function (Accuracy) definiert
def scorer_f(estimator, X, y): 
      return accuracy_score(estimator.predict(X),y)

# 3. Gittersuche parametrieren
gridSearch = GridSearchCV(model, hyper_parameters, return_train_score=True, cv=5, n_jobs=-2, scoring=scorer_f)

# 4. Modelle trainieren
gridSearch = gridSearch.fit(X_train.values, y_train)
print("\nDie Gittersuche (" + str(len(pd.DataFrame(gridSearch.cv_results_)))
      + " Kombinationen) hat " + str("%.1f" % (time.monotonic() - start_timer))
      + " Sekunden gedauert.")

### 5.1 Modelle validieren - GridSearch Ergebnisse begutachten

In der Variablen GridSearch sind nun die Ergebnisse der Gittersuche gespeichert.  

Mit dem Befahl GridSearch.cv_results_ bekommen wir die Ergebnis-Tabelle der Gittersuche (hier: besten 5 Ergebnisse):

In [ ]:
# Top 5 Ergebnisse
pd.set_option('display.max_columns', None)
pd.DataFrame(gridSearch.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

### 5.2 Modelle validieren - Modell auswählen 

Mit dem Befehl GridSearch.best_params_ bekommen wir die besten Hyperparameter der Gittersuche

In [ ]:
# Beste Kombination der Hyperparameter
gridSearch.best_params_

Nun muss das Modell mit den optimalen Hyperparametern erstellt und dann mit den Trainingsdaten trainiert werden.  
Diese Aufgabe übernimmt bereits die GridSearch-Funktion. Somit muss das beste Modell nur noch extrahiert werden, das geht mit dem Befehl GridSearch.best_estimator_

In [ ]:
# Extraktion des Modells
model = gridSearch.best_estimator_

### 5.3 Modelle validieren - Bewertung des Trainings
Wir berechnen die Genauigkeit des Modells auf den Trainingsdaten (richtige Vorhersagen/alle Vorhersagen).

In [ ]:
# Berechnung der Vorhersage basierend auf den Trainingsdaten
y_train_pred = model.predict(X_train.values)

# Berechne Genauigkeit auf den Trainingsdaten
accuracy_train = accuracy_score(y_train, y_train_pred)

# Berechne den F1-Score auf den Trainingsdaten
f1score_train = f1_score(y_train, y_train_pred)

# Ausgabe der Modellgenauigkeit
print('Ergebnis für das Training:')
print('Accuracy: \t' + str(round(accuracy_train, 4)))
print('F1-Score: \t' + str(round(f1score_train, 4)))

# Visualisierung der Konfusionsmatrix
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)
plt.grid()
plt.title('Konfusionsmatrix auf Trainingsdaten')
plt.show()

### 6.1 Modell testen & anwenden - Genauigkeit auf Testdaten und Konfusionsmatrix

Um die Qualität/Güte des Modells zu bestimmen wird dieses auf den Testdaten getestet und die Konfusionsmatrix ausgegeben.

In [ ]:
# Berechnung der Vorhersage basierend auf den Testdaten
y_test_pred = model.predict(X_test.values)

# Berechne Genauigkeit auf den Testdaten
accuracy_test = accuracy_score(y_test, y_test_pred)

# Berechne den F1-Score auf den Testdaten
f1score_test = f1_score(y_test, y_test_pred)

# Ausgabe der Modellgenauigkeit
print('Ergebnis für den Test:')
print('Accuracy: \t' + str(round(accuracy_test, 4)))
print('F1-Score: \t' + str(round(f1score_test, 4)))

# Visualisierung der Konfusionsmatrix
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
plt.grid()
plt.title('Konfusionsmatrix auf Testdaten')
plt.show()